<a href="https://colab.research.google.com/github/nagi1995/cats-vs-dogs-classification/blob/main/cats_vs_dogs_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change permission
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip "/content/train.zip" -d "./"
!unzip "/content/test1.zip" -d "./"

In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.util import plot_model
from tensorflow.keras.callbacks import *
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
import random

In [6]:
tf.__version__

'2.5.0'

In [7]:
len(os.listdir("/content/test1")), len(os.listdir("/content/train"))

(12500, 25000)

In [8]:
for i in random.sample(range(len(os.listdir("/content/train"))), 10):
  print(cv2.imread("/content/train/" + os.listdir("/content/train")[i]).shape)

(499, 341, 3)
(261, 350, 3)
(312, 285, 3)
(166, 220, 3)
(374, 500, 3)
(500, 389, 3)
(121, 181, 3)
(197, 278, 3)
(299, 400, 3)
(189, 221, 3)


In [9]:
Image_Width = 128
Image_Height = 128
Image_Size = (Image_Width, Image_Height)
Image_Channels = 3

In [10]:
dataset_path = "./train"

filenames = os.listdir(dataset_path)
categories = []
for f_name in filenames:
    
    if(f_name.split(".")[0] == "dog"):
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({'filename':filenames, 'category':categories})
df.head()

,filename,category
0,dog.10405.jpg,1
1,dog.4244.jpg,1
2,cat.3857.jpg,0
3,dog.8589.jpg,1
4,dog.8456.jpg,1


In [11]:
reduce_lr = ReduceLROnPlateau(monitor = "val_loss", 
                              factor = .4642, 
                              patience = 3, 
                              verbose = 1, 
                              min_delta = 0.001, 
                              mode = "min")
earlystop = EarlyStopping(monitor = "val_loss", 
                          patience = 10, 
                          verbose = 1, 
                          mode = "min"
                          )


In [ ]:
tf.keras.backend.clear_session()
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (Image_Width, Image_Height, Image_Channels)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        2

In [12]:
df["category"] = df["category"].replace({0:'cat', 1:'dog'})
train_df, validate_df = train_test_split(df, test_size = 0.2, random_state = 42)
train_df = train_df.reset_index(drop = True)
validate_df = validate_df.reset_index(drop = True)

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size = 15

In [13]:
train_datagen = ImageDataGenerator(rotation_range = 15, rescale = 1./255, shear_range = 0.1, zoom_range = 0.2, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1)
train_generator = train_datagen.flow_from_dataframe(train_df, dataset_path, x_col = 'filename', y_col = 'category', target_size = Image_Size, class_mode = 'categorical', batch_size = batch_size)

validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_dataframe(validate_df, dataset_path, x_col = 'filename', y_col = 'category', target_size = Image_Size, class_mode = 'categorical', batch_size = batch_size)


Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [ ]:
checkpoint = ModelCheckpoint(filepath = "./1/weights.h5", 
                             monitor = "val_loss", 
                             verbose = 1, 
                             save_best_only = True,
                             mode = "min")

callbacks_list = [checkpoint, reduce_lr, earlystop]
epochs = 150
history = model.fit(train_generator, 
                    epochs = epochs, 
                    validation_data = validation_generator, 
                    validation_steps = total_validate//batch_size, 
                    steps_per_epoch = total_train//batch_size, 
                    callbacks = callbacks_list) 


Epoch 1/150
1333/1333 [==============================] - 172s 105ms/step - loss: 1.9285 - accuracy: 0.5742 - val_loss: 0.6643 - val_accuracy: 0.6180

Epoch 00001: val_loss improved from inf to 0.66426, saving model to ./1/weights.h5
Epoch 2/150
1333/1333 [==============================] - 139s 105ms/step - loss: 0.6456 - accuracy: 0.6517 - val_loss: 0.6237 - val_accuracy: 0.6893

Epoch 00002: val_loss improved from 0.66426 to 0.62370, saving model to ./1/weights.h5
Epoch 3/150
1333/1333 [==============================] - 139s 104ms/step - loss: 0.5870 - accuracy: 0.6910 - val_loss: 0.5370 - val_accuracy: 0.7361

Epoch 00003: val_loss improved from 0.62370 to 0.53695, saving model to ./1/weights.h5
Epoch 4/150
1333/1333 [==============================] - 141s 106ms/step - loss: 0.5517 - accuracy: 0.7256 - val_loss: 0.4917 - val_accuracy: 0.7616

Epoch 00004: val_loss improved from 0.53695 to 0.49175, saving model to ./1/weights.h5
Epoch 5/150
1333/1333 [==============================] -

In [14]:
tf.keras.backend.clear_session()
model = Sequential()
model.add(SeparableConv2D(32, (3, 3), input_shape = (Image_Width, Image_Height, Image_Channels)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(SeparableConv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(SeparableConv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 126, 126, 32)      155       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 61, 61, 64)        2400      
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        2

In [15]:
checkpoint = ModelCheckpoint(filepath = "./2/weights.h5", 
                             monitor = "val_loss", 
                             verbose = 1, 
                             save_best_only = True,
                             mode = "min")

callbacks_list = [checkpoint, reduce_lr, earlystop]
epochs = 60
history = model.fit(train_generator, 
                    epochs = epochs, 
                    validation_data = validation_generator, 
                    validation_steps = total_validate//batch_size, 
                    steps_per_epoch = total_train//batch_size, 
                    callbacks = callbacks_list) 


Epoch 1/60
1333/1333 [==============================] - 177s 110ms/step - loss: 2.1659 - accuracy: 0.6026 - val_loss: 0.6370 - val_accuracy: 0.6478

Epoch 00001: val_loss improved from inf to 0.63704, saving model to ./2/weights.h5
Epoch 2/60
1333/1333 [==============================] - 146s 110ms/step - loss: 0.6195 - accuracy: 0.6732 - val_loss: 0.5653 - val_accuracy: 0.7037

Epoch 00002: val_loss improved from 0.63704 to 0.56527, saving model to ./2/weights.h5
Epoch 3/60
1333/1333 [==============================] - 146s 110ms/step - loss: 0.5848 - accuracy: 0.6895 - val_loss: 0.5341 - val_accuracy: 0.7283

Epoch 00003: val_loss improved from 0.56527 to 0.53409, saving model to ./2/weights.h5
Epoch 4/60
1333/1333 [==============================] - 145s 109ms/step - loss: 0.5622 - accuracy: 0.7049 - val_loss: 0.5518 - val_accuracy: 0.7155

Epoch 00004: val_loss did not improve from 0.53409
Epoch 5/60
1333/1333 [==============================] - 146s 110ms/step - loss: 0.5507 - accurac